### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

# Import Packages

In [ ]:
import oci
import io
import numpy as np
import matplotlib.pyplot as plt
from ads import set_auth

set_auth("resource_principal")

# Reading confusion_matrix files from Object Storage

In [1]:
signer = oci.auth.signers.get_resource_principals_signer()
object_storage = oci.object_storage.ObjectStorageClient({}, signer=signer)
namespace = object_storage.get_namespace().data
bucket_name = "filesdemo"

object_names = ["xgboost_confusion_matrix.txt", "random_forest_confusion_matrix.txt"]

matrices = []
for obj_name in object_names:
    response = object_storage.get_object(namespace, bucket_name, obj_name)
    # Read the content as string, then convert to numpy array
    content = response.data.text
    matrix = np.loadtxt(io.StringIO(content), delimiter="\t")
    matrices.append(matrix)

# Creating and Exporting the Grpah

In [ ]:
titles = ["XGBoost", "Random Forest"]

fig, axes = plt.subplots(1, 2, figsize=(12, 5), constrained_layout=True)

for i, ax in enumerate(axes):
    im = ax.imshow(matrices[i], cmap="Blues", interpolation="nearest")
    ax.set_title(titles[i])
    
    for (j, k), val in np.ndenumerate(matrices[i]):
        ax.text(k, j, f"{val:.2f}", ha='center', va='center', color='black')

fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.6)

buf = io.BytesIO()
plt.savefig(buf, format='png', bbox_inches='tight')
buf.seek(0)
plt.close(fig)  # Close the figure to free memory

signer = oci.auth.signers.get_resource_principals_signer()
object_storage = oci.object_storage.ObjectStorageClient({}, signer=signer)
namespace = object_storage.get_namespace().data

object_name = "xgboost_vs_random_forest_confusion_matrices.png"

object_storage.put_object(
    namespace_name=namespace,
    bucket_name='filesdemo',
    object_name=object_name,
    put_object_body=buf
)